In [3]:
# Generic setup block

import os, sys
import pprint
from pathlib import Path

# Set paths

project_root = Path(os.getcwd()).parent
src_path     = project_root / "src"
if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))  # so `import pyetm` works

# Hide the trackback for now

ipython = get_ipython()

def hide_traceback(exc_tuple=None, filename=None, tb_offset=None,
                   exception_only=False, running_compiled_code=False):
    etype, value, tb = sys.exc_info()
    return ipython._showtraceback(etype, value, ipython.InteractiveTB.get_exception_only(etype, value))

ipython.showtraceback = hide_traceback

# Import pyETM

from pyetm.config.settings import get_settings
from pyetm.models import Scenario

# Check if our API is ready!

print("Using ETM API at    ",       get_settings().base_url)
print("Token loaded?       ",       bool(get_settings().etm_api_token))

Using ETM API at     http://localhost:3000/api/v3
Token loaded?        True


In [4]:
from pyetm.clients import BaseClient
from pyetm.services.scenario_runners.fetch_carrier_curves import DownloadCarrierCurveRunner
import pandas as pd


scenario = Scenario.load(2690288)
client = BaseClient()

print(f"Curves from scenario with session id: {scenario.id}")
curve_name = "electricity_price"
result = DownloadCarrierCurveRunner.run(client, scenario, curve_name)
df = pd.read_csv(result.data)
print(f"Shape: {df.shape}")
print(df)

Curves from scenario with session id: 2690288
Shape: (8760, 2)
                  Time  Price (Euros)
0     2050-01-01 00:00           26.0
1     2050-01-01 01:00           26.0
2     2050-01-01 02:00           25.0
3     2050-01-01 03:00           25.0
4     2050-01-01 04:00           25.0
...                ...            ...
8755  2050-12-31 19:00           25.0
8756  2050-12-31 20:00           25.0
8757  2050-12-31 21:00           25.0
8758  2050-12-31 22:00           25.0
8759  2050-12-31 23:00           25.0

[8760 rows x 2 columns]


In [6]:
# Test fetch all curves

from pyetm.services.scenario_runners.fetch_carrier_curves import FetchAllCarrierCurvesRunner

curves = FetchAllCarrierCurvesRunner.run(client, scenario)

print(f"Downloaded {len(curves.data) if curves.data else 0} curves")
if curves.errors:
    print(f"Warnings: {curves.errors}")


Downloaded 10 curves
